# IoT Project

##### To get started

We need the following Python libraries:  
- Scapy: For parsing network packet traces
- IPy: For each IP addressing types (what is a local network address vs what is public network address, etcc)
- manuf: Look up a device manufacturer based on MAC address data

In [51]:
!pip install scapy IPy manuf pycurl StringIO

  Could not find a version that satisfies the requirement StringIO (from versions: )
No matching distribution found for StringIO


In [1]:
from scapy.all import *
from manuf import manuf

In [11]:
#help(scapy.all)

In [84]:
packets =rdpcap("./packet_capture.pcap")
p = manuf.MacParser(update=True)
# Example for testing Manufacturer look up based on MAC ID
p.get_all('84:38:35:5a:70:40')[0]
p.get_all('01:00:5e:00:00:fb')

Vendor(manuf=None, comment=None)

In [264]:
import requests
import pycurl
import pprint
import json
from io import BytesIO
import time





def extractOrg(IP):
    if IP not in public_IPs:
        url = 'https://ipinfo.io/' + IP + '?token=4b22562280ac95' 
        #c = pycurl.Curl()
        #c.setopt(c.URL, url)
        #e = BytesIO()
        #c.setopt(pycurl.WRITEFUNCTION, e.write)
        #c.perform()
        #c.close()
        #tmp = json.loads(e.getvalue())
        tmp = requests.get(url).json()
        print(tmp)
        public_IPs[IP] = tmp['org']
        time.sleep(3)
    org = public_IPs[IP]
    return (IP, org) 
    
    

In [262]:
t = extractOrg('216.58.195.243')
print(t)

('216.58.195.243', 'AS15169 Google LLC')


In [ ]:
import pandas as pd
import concurrent.futures
import requests
import time

out = []
CONNECTIONS = 100
TIMEOUT = 5
time1 = None
time2 = None

tlds = open('../data/sample_1k.txt').read().splitlines()
urls = ['http://{}'.format(x) for x in tlds[1:]]

def load_url(url, timeout):
    ans = requests.head(url, timeout=timeout)
    return ans.status_code

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = {executor.submit(load_url, url, TIMEOUT): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            data = future.result()
        except Exception as exc:
            data = str(type(exc))
        finally:
            out.append(data)

            print(str(len(out)),end="\r")

            if time1 == None:
                time1 = time.time()
            if len(out)/len(urls)>=1 and time2==None:
                time2 = time.time()

print('Took {:.2f} s'.format((time2-time1)))
print(str(pd.Series(out).value_counts()))

In [266]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scapy.all import *
from IPy import IP as IPy
import pprint

d= defaultdict(dict)

for i in range(len(packets)):
#for i in range(500):
    packet = packets[i]
    ## traffic patterns:
    # ARP: ignore for now
    
    # Ignore IPv6 for now
    if (packet.haslayer(IPv6)):
        continue 
    # DNS
    if(packet.haslayer(DNS)):
        dns_type = 'DNS'
        mac_id = packet["Ether.dst"]
        if (packet.haslayer(UDP)):
            if (packet[UDP].sport == 5353):
                dns_type = 'mDNS'
                mac_id = packet["Ether.src"]
        if(packet[DNS].haslayer(DNSRR)):
            rrname = packet[DNS][DNSRR].rrname.decode("utf-8")
            if mac_id not in d:
                d[mac_id] = {dns_type: rrname}
            else:
                if dns_type in d[mac_id]:
                    d[mac_id][dns_type] += "," + rrname
                else:
                    d[mac_id][dns_type] = rrname 
    # HTTP:
    elif(packet.haslayer(IP)):
        if (packet.haslayer(TCP)):
            mac_id = packet["Ether.src"]
            IP_src = packet["IP.src"]
            IP_dst = packet["IP.dst"]
            if (packet[TCP].dport == 443):
                # This is https, just record high level data
                if ((IPy(IP_src).iptype() == "PRIVATE") and (IPy(IP_dst).iptype() == "PUBLIC")):
                    if mac_id not in d:
                        d[mac_id] = {'HTTPS': {IP_dst}}
                    else:
                        if 'HTTPS' in d[mac_id]:
                            d[mac_id]['HTTPS'].add(IP_dst)
                        else:
                            d[mac_id]['HTTPS'] = {IP_dst}
            else:
                # This is http, record more data including the payload
                if(len(packet[TCP].payload) > 10):
                    payload = str(packet[TCP].payload)
                else:
                    payload = ""
                if ((IPy(IP_src).iptype() == "PRIVATE") and (IPy(IP_dst).iptype() == "PUBLIC")):
                    if mac_id not in d:
                        d[mac_id] = {'HTTP': {IP_dst: payload}}
                    else:
                        if 'HTTP' in d[mac_id]:
                            if IP_dst in d[mac_id]['HTTP']:
                                d[mac_id]['HTTP'][IP_dst] +=  payload
                            else:
                                d[mac_id]['HTTP'][IP_dst] = payload 
                        else:
                            d[mac_id]['HTTP']= {IP_dst : payload}  
    # Else: What did I miss
    else:
        None   
    
    # if the device is in the dataframe
    #df.loc[i] = [mac_id, p.get_all(mac_id)[0]]
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(d)

In [267]:
dframe = pd.DataFrame.transpose(pd.DataFrame(d))
dframe

,DNS,HTTP,HTTPS,mDNS
00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN
28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","{173.194.203.95, 216.58.194.165, 52.201.158.18...",NaN
40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0..."
44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...",{52.46.136.77},NaN
5c:aa:fd:4c:87:a0,NaN,NaN,{54.208.201.219},NaN
5c:aa:fd:4c:92:86,NaN,NaN,{54.163.161.36},NaN
78:28:ca:03:80:0c,NaN,{'151.101.40.246': 'b'GET /audio/7fd0fe06a28db...,{107.22.76.192},NaN
84:38:35:5a:70:40,"googleads.g.doubleclick.net.,cm.g.doubleclick....","{'185.217.0.110': '', '104.154.127.116': 'b'\x...","{172.217.6.65, 172.217.6.33, 199.59.150.42, 74...",NaN
88:15:44:aa:47:2e,"yahoo.com.,yahoo.com.,yahoo.com.,0.1.0.0.0.0.0...",NaN,NaN,NaN
88:71:e5:d2:73:4b,NaN,{'104.154.126.239': 'b'\xb2\xf4\xdf\xc1\xac\xe...,"{52.46.132.50, 34.236.127.105}",NaN


In [299]:
import pandas as pd
import concurrent.futures
import requests
import time

out = {}
CONNECTIONS = 100
TIMEOUT = 5
time1 = None
time2 = None

urls = set()
dframe['HTTPS'].dropna().apply(lambda x: urls.update(x))

def load_url(url, timeout):
    #url_full = 'https://ipinfo.io/' + url + '?token=4b22562280ac95'
    url_full = 'https://api.iptoasn.com/v1/as/ip/' + url
    ans = requests.get(url_full, timeout=timeout).json()
    return (url, ans['as_description'])

with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = {executor.submit(load_url, url, TIMEOUT): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            url, as_decription = future.result()
            out[url] = as_decription
        except Exception as exc:
            data = str(type(exc))
            print(future)
            print(data)
        #finally:
            #out.append(data)
  


In [318]:
def lookup(url_list):
    if pd.isna(url_list):
        return url_list
    else:
        return [ url +' : '+ out[url] for url in url_list]

dframe['HTTPS'] = dframe['HTTPS'].apply(lambda x: lookup(x))


In [319]:
dframe

,DNS,HTTP,HTTPS,mDNS
00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN
28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","[173.194.203.95 : GOOGLE - Google LLC, 216.58....",NaN
40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0..."
44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...","[52.46.136.77 : AMAZON-02 - Amazon.com, Inc.]",NaN
5c:aa:fd:4c:87:a0,NaN,NaN,"[54.208.201.219 : AMAZON-AES - Amazon.com, Inc.]",NaN
5c:aa:fd:4c:92:86,NaN,NaN,"[54.163.161.36 : AMAZON-AES - Amazon.com, Inc.]",NaN
78:28:ca:03:80:0c,NaN,{'151.101.40.246': 'b'GET /audio/7fd0fe06a28db...,"[107.22.76.192 : AMAZON-AES - Amazon.com, Inc.]",NaN
84:38:35:5a:70:40,"googleads.g.doubleclick.net.,cm.g.doubleclick....","{'185.217.0.110': '', '104.154.127.116': 'b'\x...","[172.217.6.65 : GOOGLE - Google LLC, 172.217.6...",NaN
88:15:44:aa:47:2e,"yahoo.com.,yahoo.com.,yahoo.com.,0.1.0.0.0.0.0...",NaN,NaN,NaN
88:71:e5:d2:73:4b,NaN,{'104.154.126.239': 'b'\xb2\xf4\xdf\xc1\xac\xe...,"[52.46.132.50 : AMAZON-02 - Amazon.com, Inc., ...",NaN


# Global list of device manufacturers 

Organization Unique Identifier, aka OUI data from the IEEE website:

In [320]:
OUI_df = pd.read_csv('oui.csv')
OUI_df.head()

,Registry,Assignment,Organization Name,Organization Address
0,MA-L,E043DB,"Shenzhen ViewAt Technology Co.,Ltd.","9A,Microprofit,6th Gaoxin South Road, High-Tec..."
1,MA-L,2405F5,Integrated Device Technology (Malaysia) Sdn. Bhd.,"Phase 3, Bayan Lepas FIZ Bayan Lepas Penang MY..."
2,MA-L,3CD92B,Hewlett Packard,11445 Compaq Center Drive Houston US 77070
3,MA-L,9C8E99,Hewlett Packard,11445 Compaq Center Drive Houston US 77070
4,MA-L,B499BA,Hewlett Packard,11445 Compaq Center Drive Houston US 77070


In [321]:
dframe.index.name = 'MacID'
dframe.reset_index(inplace=True)
dframe.head()

,MacID,DNS,HTTP,HTTPS,mDNS
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","[173.194.203.95 : GOOGLE - Google LLC, 216.58....",NaN
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0..."
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...","[52.46.136.77 : AMAZON-02 - Amazon.com, Inc.]",NaN
4,5c:aa:fd:4c:87:a0,NaN,NaN,"[54.208.201.219 : AMAZON-AES - Amazon.com, Inc.]",NaN


In [322]:
# Let's extract the Organization Unique Identifier, aka OUI from the MAC address
# which is simply the first 6 alphanumerical characters
dframe['OUI'] = dframe['MacID'].apply(lambda x: ''.join(x.upper().split(':')[0:3]))

# Merging the manufacturer information from the master database (downloaded from the IEEE website)
# with our dataframe
dframe = dframe.merge(OUI_df[list(['Assignment', 'Organization Name'])], 
             left_on = 'OUI',  right_on= 'Assignment', how = 'left')\
            [list(['MacID', 'DNS', 'HTTP', 'HTTPS', 'mDNS', 'OUI', 'Organization Name'])]
dframe.head()

,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN,00180A,Cisco Meraki
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","[173.194.203.95 : GOOGLE - Google LLC, 216.58....",NaN,28F076,"Apple, Inc."
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,"Apple, Inc."
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...","[52.46.136.77 : AMAZON-02 - Amazon.com, Inc.]",NaN,44650D,Amazon Technologies Inc.
4,5c:aa:fd:4c:87:a0,NaN,NaN,"[54.208.201.219 : AMAZON-AES - Amazon.com, Inc.]",NaN,5CAAFD,"Sonos, Inc."


## Device labeling  
The packet traces we are using come from my home network. I know what these devices are. I am going to import the device information for these devices and merge it with the network traffic dataframe we are using.

In [323]:
known_devices = pd.read_csv('known_devices.csv')
known_devices.head()

,Description,Last seen,Usage,OS,IPv4 address,Policy,MAC address,Manufacturer,Device Type
0,Daghan deskphone,6/6/18 11:29,9.626191e+05,Meraki,192.168.1.236,normal,00:18:0a:12:2f:2d,Meraki,Phone
1,Nest,6/6/18 11:30,8.349500e+04,Raspberry Pi,192.168.1.235,normal,18:b4:30:0d:87:18,Nest Labs,Thermostat
2,iPhone,5/19/18 18:11,4.825600e+06,Apple iPhone,192.168.1.9,Privacy please,18:f6:43:57:fc:8f,Apple,iPhone
3,Daghan-Home,6/6/18 11:30,3.126772e+07,Mac OS X 10.13,192.168.1.233,normal,28:f0:76:31:d3:58,Apple,Computer
4,DaghanspleWatch,6/5/18 5:43,2.672940e+05,iOS,192.168.1.7,normal,30:63:6b:65:e4:04,Apple,Watch


In [324]:
known_devices['OUI'] = known_devices['MAC address'].apply(lambda x: ''.join(x.upper().split(':')[0:3]))
known_devices['Manufacturer Device Type'] = ([' '.join(row) for row in 
                        zip(known_devices["Manufacturer"],known_devices["Device Type"])])
known_devices

,Description,Last seen,Usage,OS,IPv4 address,Policy,MAC address,Manufacturer,Device Type,OUI,Manufacturer Device Type
0,Daghan deskphone,6/6/18 11:29,9.626191e+05,Meraki,192.168.1.236,normal,00:18:0a:12:2f:2d,Meraki,Phone,00180A,Meraki Phone
1,Nest,6/6/18 11:30,8.349500e+04,Raspberry Pi,192.168.1.235,normal,18:b4:30:0d:87:18,Nest Labs,Thermostat,18B430,Nest Labs Thermostat
2,iPhone,5/19/18 18:11,4.825600e+06,Apple iPhone,192.168.1.9,Privacy please,18:f6:43:57:fc:8f,Apple,iPhone,18F643,Apple iPhone
3,Daghan-Home,6/6/18 11:30,3.126772e+07,Mac OS X 10.13,192.168.1.233,normal,28:f0:76:31:d3:58,Apple,Computer,28F076,Apple Computer
4,DaghanspleWatch,6/5/18 5:43,2.672940e+05,iOS,192.168.1.7,normal,30:63:6b:65:e4:04,Apple,Watch,30636B,Apple Watch
5,daghan-macbook-8,6/6/18 11:21,2.605438e+07,Mac OS X 10.13,192.168.1.21,Privacy please,34:36:3b:7f:3f:a0,Apple,Computer,34363B,Apple Computer
6,Daghan-Home,6/6/18 11:29,2.155549e+05,Mac OS X,192.168.1.224,normal,38:c9:86:40:7c:a6,Apple,Computer,38C986,Apple Computer
7,Apple-TV-3,6/3/18 19:35,1.000000e+00,iOS,192.168.1.247,normal,40:cb:c0:bc:36:7c,Apple,TV,40CBC0,Apple TV
8,Apple-TV-3,6/6/18 11:29,6.434089e+05,iOS,192.168.1.249,normal,40:cb:c0:bc:36:7e,Apple,TV,40CBC0,Apple TV
9,44:65:0d:66:72:60,6/6/18 1:22,3.573050e+05,Generic Linux,192.168.1.19,normal,44:65:0d:66:72:60,Amazon Technologies,Echo,44650D,Amazon Technologies Echo


In [325]:
dframe.head()

,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN,00180A,Cisco Meraki
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","[173.194.203.95 : GOOGLE - Google LLC, 216.58....",NaN,28F076,"Apple, Inc."
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,"Apple, Inc."
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...","[52.46.136.77 : AMAZON-02 - Amazon.com, Inc.]",NaN,44650D,Amazon Technologies Inc.
4,5c:aa:fd:4c:87:a0,NaN,NaN,"[54.208.201.219 : AMAZON-AES - Amazon.com, Inc.]",NaN,5CAAFD,"Sonos, Inc."


## Merging the device labels with the packet trace dataframe

In [326]:
# We are going to merge the labels from the known devices (my home) dataframe
dframe = dframe.merge(known_devices[list(['MAC address', 'Device Type','Manufacturer Device Type'])], 
             left_on = 'MacID',  right_on= 'MAC address', how = 'left').drop('MAC address', axis=1)

In [327]:
#let's drop NAs (rows without any label)
#I'll investigate what's going on later
dframe = dframe.dropna(0, subset = ['Manufacturer Device Type'])
dframe

,MacID,DNS,HTTP,HTTPS,mDNS,OUI,Organization Name,Device Type,Manufacturer Device Type
0,00:18:0a:12:2f:2d,NaN,{'199.231.78.217': 'b'\x17\x03\x01\x02 0<_7\xf...,NaN,NaN,00180A,Cisco Meraki,Phone,Meraki Phone
1,28:f0:76:31:d3:58,clients6.google.com.,"{'104.154.127.3': 'b""\x93\x938\x1f\x0f\xe5\xb7...","[173.194.203.95 : GOOGLE - Google LLC, 216.58....",NaN,28F076,"Apple, Inc.",Computer,Apple Computer
2,40:cb:c0:bc:36:7e,NaN,NaN,NaN,"40CBC0BC367E@Apple TV._raop._tcp.local.,40CBC0...",40CBC0,"Apple, Inc.",TV,Apple TV
3,44:65:0d:90:60:3a,NaN,"{'104.154.127.60': 'b""M]'3\x13\x8c\x93\xbaoO\x...","[52.46.136.77 : AMAZON-02 - Amazon.com, Inc.]",NaN,44650D,Amazon Technologies Inc.,Echo,Amazon Technologies Echo
4,5c:aa:fd:4c:87:a0,NaN,NaN,"[54.208.201.219 : AMAZON-AES - Amazon.com, Inc.]",NaN,5CAAFD,"Sonos, Inc.",Speaker,Sonos Speaker
5,5c:aa:fd:4c:92:86,NaN,NaN,"[54.163.161.36 : AMAZON-AES - Amazon.com, Inc.]",NaN,5CAAFD,"Sonos, Inc.",Speaker,Sonos Speaker
6,78:28:ca:03:80:0c,NaN,{'151.101.40.246': 'b'GET /audio/7fd0fe06a28db...,"[107.22.76.192 : AMAZON-AES - Amazon.com, Inc.]",NaN,7828CA,"Sonos, Inc.",Speaker,Sonos Speaker
7,84:38:35:5a:70:40,"googleads.g.doubleclick.net.,cm.g.doubleclick....","{'185.217.0.110': '', '104.154.127.116': 'b'\x...","[172.217.6.65 : GOOGLE - Google LLC, 172.217.6...",NaN,843835,"Apple, Inc.",Computer,Apple Computer
9,88:71:e5:d2:73:4b,NaN,{'104.154.126.239': 'b'\xb2\xf4\xdf\xc1\xac\xe...,"[52.46.132.50 : AMAZON-02 - Amazon.com, Inc., ...",NaN,8871E5,Amazon Technologies Inc.,Echo,Amazon Technologies Echo
10,b4:7c:9c:31:b4:df,"spectrum.s3.amazonaws.com.,ntp-g7g.amazon.com....","{'93.184.216.34': '', '52.216.17.168': '', '52...","[54.239.27.11 : AMAZON-02 - Amazon.com, Inc., ...","linux-6.local.,linux-6.local.,Android.local.,l...",B47C9C,Amazon Technologies Inc.,Echo,Amazon Technologies Echo


In [340]:
dframe.iloc[1]['HTTPS']

['173.194.203.95 : GOOGLE - Google LLC',
 '216.58.194.165 : GOOGLE - Google LLC',
 '52.201.158.18 : AMAZON-AES - Amazon.com, Inc.',
 '18.204.151.40 : AMAZON-AES - Amazon.com, Inc.',
 '172.217.6.39 : GOOGLE - Google LLC',
 '172.217.164.106 : GOOGLE - Google LLC',
 '104.86.220.69 : CMCS - Comcast Cable Communications, LLC']

# Ignore everything below

In [211]:
import pycurl
import pprint
import json
from io import BytesIO


c = pycurl.Curl()
c.setopt(c.URL, 'https://ipinfo.io/216.58.195.243?token=4b22562280ac95')

e = BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = json.loads(e.getvalue())
pprint.pprint(htmlString)

{'city': 'Mountain View',
 'country': 'US',
 'hostname': 'sfo03s06-in-f19.1e100.net',
 'ip': '216.58.195.243',
 'loc': '37.4192,-122.0570',
 'org': 'AS15169 Google LLC',
 'postal': '94043',
 'region': 'California'}


In [210]:
c = pycurl.Curl()
c.setopt(c.URL, 'https://api.iptoasn.com/v1/as/ip/216.58.195.243')
e = io.BytesIO()
c.setopt(pycurl.WRITEFUNCTION, e.write)
c.perform()
c.close()
htmlString = e.getvalue().decode('UTF-8')
print(htmlString)

{"announced":true,"as_country_code":"US","as_description":"GOOGLE - Google LLC","as_number":15169,"first_ip":"216.58.192.0","ip":"216.58.195.243","last_ip":"216.58.199.255"}


In [223]:
import pprint
import requests

dictionary = requests.get('https://ipinfo.io/216.58.195.243?token=4b22562280ac95').json()
tmp = {k: dictionary[k] for k in ('org', 'hostname')}
pprint.pprint(tmp)



#dictionary = requests.get('https://api.iptoasn.com/v1/as/ip/216.58.195.243').json()["hostname"]
#pprint.pprint(dictionary)



{'hostname': 'sfo03s06-in-f243.1e100.net', 'org': 'AS15169 Google LLC'}
